In [ ]:
import os
import random
import math
from datetime import datetime
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms, datasets

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [ ]:
config = {
    "img_size": 32,
    "patch_size": 4,
    "in_chans": 3,
    "num_classes": 10,
    "embed_dim": 192,
    "depth": 8,
    "num_heads": 8,
    "mlp_ratio": 4.0,
    "dropout": 0.1,
    "attn_dropout": 0.1,
    "lr": 3e-4,
    "weight_decay": 0.05,
    "batch_size": 128,
    "epochs": 50,
    "grad_clip": None,
    "warmup_steps": 500,
}



In [ ]:
mean = (0.4914, 0.4822, 0.4465)
std  = (0.2470, 0.2435, 0.2616)

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandAugment(num_ops=2, magnitude=9),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])


test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

data_root = "./data"
train_dataset = datasets.CIFAR10(root=data_root, train=True, download=True, transform=train_transform)
test_dataset  = datasets.CIFAR10(root=data_root, train=False, download=True, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_dataset,  batch_size=config["batch_size"], shuffle=False, num_workers=2, pin_memory=True)

print(f"Train size: {len(train_dataset)}, Test size: {len(test_dataset)}")


100%|██████████| 170M/170M [00:03<00:00, 43.5MB/s]


Train size: 50000, Test size: 10000


In [ ]:
# Cell: MixUp helpers
import torch.nn.functional as F
import numpy as np

def mixup_data(x, y, alpha=0.8):
    """Returns mixed inputs, pairs of targets, and lambda"""
    if alpha <= 0:
        return x, y, 1.0, y, 1.0
    lam = np.random.beta(alpha, alpha)
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(x.device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, lam, y_b

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)


In [ ]:
class PatchEmbed(nn.Module):
    def __init__(self, img_size=32, patch_size=4, in_chans=3, embed_dim=128):
        super().__init__()
        assert img_size % patch_size == 0, "img_size must be divisible by patch_size"
        self.patch_size = patch_size
        self.num_patches = (img_size // patch_size) ** 2
        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)
    def forward(self, x):
        # x: (B, C, H, W)
        x = self.proj(x)
        B, C, H, W = x.shape
        x = x.flatten(2).transpose(1, 2)
        return x

In [ ]:
_pe = PatchEmbed(img_size=config["img_size"], patch_size=config["patch_size"], in_chans=config["in_chans"], embed_dim=config["embed_dim"])
dummy = torch.randn(2, 3, 32, 32)
out = _pe(dummy)
print("PatchEmbed output shape:", out.shape)

PatchEmbed output shape: torch.Size([2, 64, 192])


In [ ]:
class MLP(nn.Module):
    def __init__(self, in_features, hidden_features=None, dropout=0.0):
        super().__init__()
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(hidden_features, in_features)
        self.drop = nn.Dropout(dropout)
    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

In [ ]:
class TransformerEncoderBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, mlp_ratio=4.0, dropout=0.1, attn_dropout=0.1):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_dim)
        self.attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=attn_dropout, batch_first=True)
        self.drop1 = nn.Dropout(dropout)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.mlp = MLP(embed_dim, int(embed_dim * mlp_ratio), dropout)
    def forward(self, x):
        x_norm = self.norm1(x)
        attn_out, _ = self.attn(x_norm, x_norm, x_norm, need_weights=False)
        x = x + self.drop1(attn_out)
        x_norm = self.norm2(x)
        x = x + self.mlp(x_norm)
        return x

In [ ]:
class ViT(nn.Module):
    def __init__(self, img_size=32, patch_size=4, in_chans=3, num_classes=10,
                 embed_dim=128, depth=6, num_heads=8, mlp_ratio=4.0, dropout=0.1, attn_dropout=0.1):
        super().__init__()
        self.patch_embed = PatchEmbed(img_size=img_size, patch_size=patch_size, in_chans=in_chans, embed_dim=embed_dim)
        num_patches = self.patch_embed.num_patches
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, embed_dim))
        self.pos_drop = nn.Dropout(p=dropout)
        self.blocks = nn.ModuleList([
            TransformerEncoderBlock(embed_dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, dropout=dropout, attn_dropout=attn_dropout)
            for _ in range(depth)
        ])
        self.norm = nn.LayerNorm(embed_dim)
        self.head = nn.Linear(embed_dim, num_classes)

        nn.init.trunc_normal_(self.pos_embed, std=0.02)
        nn.init.trunc_normal_(self.cls_token, std=0.02)
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.trunc_normal_(m.weight, std=0.02)
            if m.bias is not None:
                nn.init.zeros_(m.bias)
        elif isinstance(m, nn.LayerNorm):
            nn.init.zeros_(m.bias)
            nn.init.ones_(m.weight)

    def forward(self, x):
        B = x.shape[0]
        x = self.patch_embed(x)
        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.pos_embed
        x = self.pos_drop(x)
        for blk in self.blocks:
            x = blk(x)
        x = self.norm(x)
        cls_out = x[:, 0]
        logits = self.head(cls_out)
        return logits

In [ ]:
class BatchWarmupCosineScheduler:
    def __init__(self, optimizer, base_lr, total_steps, warmup_steps=0, min_lr=0.0):
        self.optimizer = optimizer
        self.base_lr = base_lr
        self.total_steps = max(1, total_steps)
        self.warmup_steps = max(0, warmup_steps)
        self.min_lr = min_lr
        self.step_num = 0

    def step(self):
        self.step_num += 1
        if self.step_num <= self.warmup_steps and self.warmup_steps > 0:
            lr = self.base_lr * float(self.step_num) / float(max(1, self.warmup_steps))
        else:
            progress = float(self.step_num - self.warmup_steps) / float(max(1, self.total_steps - self.warmup_steps))
            progress = min(max(progress, 0.0), 1.0)
            lr = self.min_lr + 0.5 * (self.base_lr - self.min_lr) * (1.0 + math.cos(math.pi * progress))

        for pg in self.optimizer.param_groups:
            pg['lr'] = lr

    def get_lr(self):
        return self.optimizer.param_groups[0]['lr']


In [ ]:
model = ViT(img_size=config["img_size"], patch_size=config["patch_size"], in_chans=config["in_chans"],
            num_classes=config["num_classes"], embed_dim=config["embed_dim"], depth=config["depth"],
            num_heads=config["num_heads"], mlp_ratio=config["mlp_ratio"], dropout=config["dropout"], attn_dropout=config["attn_dropout"])
print("Model params (M):", sum(p.numel() for p in model.parameters())/1e6)
dummy = torch.randn(4, 3, 32, 32)
print("Forward pass shape:", model(dummy).shape)
model.to(device)

Model params (M): 3.583306
Forward pass shape: torch.Size([4, 10])


ViT(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 192, kernel_size=(4, 4), stride=(4, 4))
  )
  (pos_drop): Dropout(p=0.1, inplace=False)
  (blocks): ModuleList(
    (0-7): 8 x TransformerEncoderBlock(
      (norm1): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
      (attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=192, out_features=192, bias=True)
      )
      (drop1): Dropout(p=0.1, inplace=False)
      (norm2): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
      (mlp): MLP(
        (fc1): Linear(in_features=192, out_features=768, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=768, out_features=192, bias=True)
        (drop): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (norm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
  (head): Linear(in_features=192, out_features=10, bias=True)
)

In [ ]:
def accuracy(output, target, topk=(1,)):
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)
        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))
        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append((correct_k.mul_(100.0 / batch_size)).item())
        return res

from torch.cuda.amp import autocast, GradScaler
scaler = GradScaler()

def train_one_epoch_mixup(model, dataloader, criterion, optimizer, device, epoch, scheduler=None, grad_clip=None, mixup_alpha=0.8, use_amp=True):
    model.train()
    running_loss = 0.0
    running_acc = 0.0
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc=f"Train Epoch {epoch}")
    for i, (images, targets) in pbar:
        images = images.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)

        if mixup_alpha > 0:
            images, targets_a, lam, targets_b = mixup_data(images, targets, mixup_alpha)
        else:
            targets_a, targets_b, lam = targets, targets, 1.0

        optimizer.zero_grad()
        if use_amp:
            with autocast():
                outputs = model(images)
                loss = mixup_criterion(criterion, outputs, targets_a, targets_b, lam)
            scaler.scale(loss).backward()
            if grad_clip:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(images)
            loss = mixup_criterion(criterion, outputs, targets_a, targets_b, lam)
            loss.backward()
            if grad_clip:
                torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            optimizer.step()

        if scheduler is not None:
            if hasattr(scheduler, "step"):
                scheduler.step()
            elif callable(scheduler):
                scheduler()

        with torch.no_grad():
            _, preds = outputs.topk(1, dim=1)
            preds = preds.squeeze(1)
            correct = (preds == targets_a).float() * lam + (preds == targets_b).float() * (1 - lam)
            acc1 = correct.mean().item() * 100.0

        running_loss = (running_loss * i + loss.item()) / (i + 1)
        running_acc = (running_acc * i + acc1) / (i + 1)
        pbar.set_postfix(loss=running_loss, acc=running_acc)
    return running_loss, running_acc


def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0.0
    total_acc = 0.0
    with torch.no_grad():
        pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc="Eval")
        for i, (images, targets) in pbar:
            images = images.to(device, non_blocking=True)
            targets = targets.to(device, non_blocking=True)
            outputs = model(images)
            loss = criterion(outputs, targets)
            acc1 = accuracy(outputs, targets, topk=(1,))[0]
            total_loss = (total_loss * i + loss.item()) / (i + 1)
            total_acc = (total_acc * i + acc1) / (i + 1)
            pbar.set_postfix(loss=total_loss, acc=total_acc)
    return total_loss, total_acc


/tmp/ipython-input-1705434590.py:16: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [ ]:
class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, smoothing=0.1):
        super().__init__()
        self.smoothing = smoothing
    def forward(self, x, target):
        log_probs = F.log_softmax(x, dim=-1)
        nll = -log_probs.gather(dim=-1, index=target.unsqueeze(1)).squeeze(1)
        smooth_loss = -log_probs.mean(dim=-1)
        loss = (1.0 - self.smoothing) * nll + self.smoothing * smooth_loss
        return loss.mean()

In [ ]:
from torch.optim import AdamW

model = model.to(device)
criterion = LabelSmoothingCrossEntropy(smoothing=0.1)
optimizer = AdamW(model.parameters(), lr=config["lr"], weight_decay=config["weight_decay"])

total_steps = len(train_loader) * config["epochs"]
warmup_steps = config.get("warmup_steps", 500)

scheduler = BatchWarmupCosineScheduler(optimizer, base_lr=config["lr"], total_steps=total_steps, warmup_steps=warmup_steps, min_lr=1e-6)

best_acc = 0.0
history = {"train_loss": [], "train_acc": [], "test_loss": [], "test_acc": []}

for epoch in range(1, config["epochs"] + 1):
    train_loss, train_acc = train_one_epoch_mixup(model, train_loader, criterion, optimizer, device, epoch, scheduler=scheduler, grad_clip=config.get("grad_clip"))
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)
    history["train_loss"].append(train_loss); history["train_acc"].append(train_acc)
    history["test_loss"].append(test_loss); history["test_acc"].append(test_acc)

    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), "best_vit_cifar10.pth")
        print(f" => Saved best model with acc: {best_acc:.2f}%")
    print(f"Epoch {epoch} summary -> train_acc: {train_acc:.2f}%, test_acc: {test_acc:.2f}%")



Train Epoch 1:   0%|          | 0/391 [00:00<?, ?it/s]/tmp/ipython-input-1705434590.py:34: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Eval: 100%|██████████| 79/79 [00:04<00:00, 18.21it/s, acc=35.7, loss=1.91]


 => Saved best model with acc: 35.71%
Epoch 1 summary -> train_acc: 20.21%, test_acc: 35.71%


Eval: 100%|██████████| 79/79 [00:03<00:00, 21.54it/s, acc=41.1, loss=1.78]


 => Saved best model with acc: 41.14%
Epoch 2 summary -> train_acc: 27.54%, test_acc: 41.14%


Eval: 100%|██████████| 79/79 [00:02<00:00, 26.63it/s, acc=47.1, loss=1.64]


...

Epoch 48 summary -> train_acc: 56.76%, test_acc: 78.51%



Eval: 100%|██████████| 79/79 [00:02<00:00, 27.15it/s, acc=78.5, loss=0.988]

Epoch 49 summary -> train_acc: 55.82%, test_acc: 78.54%



Eval: 100%|██████████| 79/79 [00:03<00:00, 25.94it/s, acc=78.5, loss=0.988]

Epoch 50 summary -> train_acc: 56.59%, test_acc: 78.48%


In [ ]:
best_path = "best_vit_cifar10.pth"
if os.path.exists(best_path):
    model.load_state_dict(torch.load(best_path, map_location=device))
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)
    print(f"Best model test accuracy: {test_acc:.2f}%")
else:
    print("No saved model found, evaluate current model instead.")
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)
    print(f"Current model test accuracy: {test_acc:.2f}%")


Eval: 100%|██████████| 79/79 [00:02<00:00, 27.38it/s, acc=78.5, loss=0.99]

Best model test accuracy: 78.54%
